# Configurações

In [10]:
!pip -q install google-genai

In [11]:
!pip install -q google-adk

In [12]:
import os
# Importa userdata somente se estiver no ambiente Colab
try:
    from google.colab import userdata
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except ImportError:
    # Caso não esteja no Colab, a chave deve ser configurada de outra forma (ex: variável de ambiente)
    print("Não está no ambiente Google Colab. Certifique-se de que a variável de ambiente GOOGLE_API_KEY está configurada.")
    # Para rodar localmente, você pode descomentar a linha abaixo e colar sua chave (NÃO RECOMENDADO PARA COMPARTILHAR)
    # os.environ["GOOGLE_API_KEY"] = "SUA_CHAVE_API_AQUI"


# Configura o cliente da SDK do Gemini
from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash" # Modelo a ser usado

# Importações adicionais e configurações para os agentes
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
# Importa display e Markdown somente se estiver no ambiente Colab
try:
    from IPython.display import display, Markdown
except ImportError:
     # Define funções dummy caso não esteja no Colab para evitar erros
     def display(obj):
         print(obj) # Simplesmente imprime para o console
     class Markdown:
         def __init__(self, text):
             self.text = text
         def __str__(self):
             return self.text


import requests
import warnings
warnings.filterwarnings("ignore") # Ignora alguns avisos chatos

# Funções Auxiliares

In [13]:
# --- Funções Auxiliares ---

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown (funciona no Colab)
def to_markdown(text):
  text = text.replace('•', '  *')
  # Adiciona uma indentação para destacar a resposta no Colab
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))



# Código principal

In [17]:
# --- Definição dos Agentes ---

# Agente 1: Buscador de locais
def agente_buscador(input_inicial):
  buscador = Agent(
      name = "agente_buscador",
      model = MODEL_ID,
      description="Agente que busca locais de interesse e restaurantes na cidade do usuário",
      tools=[google_search],
      instruction="""
      Você é um assistente de busca especializado em encontrar locais para encontros (dates).
      Sua tarefa principal é coletar uma lista inicial robusta e relevante de opções na cidade especificada,
      que servirá como base para o próximo agente (Refinador de Encontro) criar sugestões de roteiros.
      Utilizando principalmente a ferramenta de busca do google (google_search) para informações contextuais adicionais (ex: verificar o "buzz", tipo de ambiente
      não explícito no `Maps`), procure por locais adequados para um encontro na cidade fornecida pelo usuário.
      Recupere uma lista diversificada com aproximadamente **15 restaurantes** e **15 locais de interesse**
      (como parques, museus, bares com ambiente para encontro, atividades culturais, pontos turísticos, etc.) que se encaixem nos critérios de orçamento e horário do usuário.
      A busca deve estar limitada a 20km do local indicado
      """
  )
  entrada_do_agente_buscador = f"Preferências do usuário: {input_inicial}"
  locais_preferenciais = call_agent(buscador, entrada_do_agente_buscador)
  return locais_preferenciais

# Agente 2: Refinador de encontro
def agente_refinador(input_inicial, locais_preferenciais):
    refinador = Agent(
        name="agente_refinador",
        model=MODEL_ID,
        instruction="""
         Você é um Refinador de Encontros, especialista em criar experiências personalizadas e agradáveis.
            Com base nas preferências, restrições e orçamento fornecidos pelo usuário, você deve:
            - Receber as seguintes informações do usuário:
              - Atividades que o usuário ou a pessoa a ser agradada *não* gostam de fazer.
              - Restrições ou preferências alimentares (o que não gosta ou não pode comer).
              - Orçamento disponível para o passeio.
              - Horário de início e/ou duração desejada para o encontro.
              - Considerar a área ou cidade relevante para a busca.
            - Usar a resposta encontrada pelo agente_buscador para selecionar os locais
            - Combinar os resultados da busca para criar *três opções distintas* de encontros, onde cada opção inclua:
              - Um restaurante.
              - Um ou dois locais de interesse próximos ou logicamente conectáveis ao restaurante.
            - Para cada uma das três opções, você deve também:
              - Sugerir uma rota lógica ou sequência de locais para o encontro (ex: "começar no parque, depois ir ao restaurante").
            - Oferecer ideias de presentes ou pequenos mimos que poderiam agradar a pessoa, NÃO OFEREÇA IDEIAS DE PRESENTES PARA O PRÓPRIO USUÁRIO, SOMENTE PARA A COMPANHIA DELE, considerando o contexto dos locais escolhidos
             e o orçamento geral (sem necessariamente detalhar o preço do presente, mas sim o tipo)
             - Ao apresentar as três opções, certificar-se de que cada uma delas respeita *todas* as restrições
              e preferências fornecidas pelo usuário (não fazer o que não gostam, respeitar a dieta/preferências alimentares,
              manter-se dentro do orçamento e compatível com o horário).
              - Apresentar as três opções de forma clara e numerada, detalhando em cada uma: o restaurante (nome, tipo, endereço), os locais de interesse (nome, tipo, endereço),
              a sugestão de rota e as ideias de presentes. Destaque brevemente como cada opção se alinha com os requisitos do usuário.
        """,
        description="Agente que refina o encontro e informações encontradas pelo usuário.",
        tools=[google_search]
    )
    entrada_do_agente_refinador = f"Preferências:{input_inicial}\nLocais buscados: {locais_preferenciais}"
    plano_de_encontro = call_agent(refinador, entrada_do_agente_refinador)
    return plano_de_encontro

# Agente 3: Planejador de encontro
def agente_planejador(input_inicial, plano_de_encontro):
    planejador = Agent(
        name="agente_planejador",
        model= MODEL_ID,
        instruction="""
            Você é o Planejador Final de Encontros, o especialista em avaliar, selecionar e apresentar o plano de encontro ideal com carinho e detalhes.
            Você receberá como entrada as 3 opções de planos de encontro que foram refinados e sugeridos pelo agente anterior, cada um contendo restaurante,
            locais de interesse, sugestão de rota e ideias de presentes.
            Sua tarefa principal é analisar cuidadosamente essas 3 opções, considerando os critérios originais do usuário que guiaram a criação dessas opções
            (horário, orçamento, coisas que não gosta/não pode, gostos/preferências gerais) para identificar qual delas é o *melhor match*
            e oferece a experiência mais harmoniosa, agradável e alinhada com o que o usuário busca.
            Após escolher o plano ideal, você deve:
              - Criar uma introdução breve, calorosa e amigável explicando *por quê* este plano específico foi selecionado como o 'perfeito' para o usuário,
              conectando-se com os gostos e necessidades dele. Use um tom pessoal e positivo. ✨
              - Detalhar o plano escolhido passo a passo, como um roteiro claro.
              - Para cada etapa do plano (ex: "Parque [Nome]", "Restaurante [Nome]", "Visita ao [Local de Interesse]"),
              indicar o *momento* ou *horário* apropriado dentro da sequência do encontro (ex: "Comece por volta das 15h no...",
              "Em seguida, por volta das 17h, dirija-se ao...","Para fechar a noite, planeje estar no... a partir das 19h").
              - Fornecer mais detalhes e informações interessantes sobre cada local incluído no plano escolhido.
              - Usar a ferramenta de busca do google (google_search) para pesquisar informações adicionais relevantes,
              curiosidades ou destaques sobre esses locais (ex: um prato famoso no restaurante, a história do parque, uma exposição temporária no museu).
              - Incluir as informações obtidas com o (google_search) *imediatamente após* a menção do local ou detalhe, sempre *entre parênteses*
              (Ex: Visite o Museu de Arte Moderna (MAC) (Famoso por sua arquitetura de Niemeyer e vista panorâmica da cidade)).
              - Incluir as ideias de presentes que foram sugeridas para o plano escolhido em um tópico separado ou integrado à descrição do plano,
              mas destaque-o para não ficar perdido no meio do texto.
              - Garantir que todo o texto seja apresentado de forma simples, acessível e que mantenha a conexão com o usuário ao longo da leitura.
              - Incluir emojis relevantes e divertidos para tornar a mensagem mais leve e expressiva. 😄🗺️🥂🎁
              - Finalizar o texto com uma mensagem genuinamente encorajadora e positiva para o usuário, transmitindo confiança e desejando um encontro maravilhoso. ❤️

              ***********Você não deve:****************
              - Dizer que escolheu uma entre três opções. O usuário não precisa saber disso.
              - Não deve dizer que criou três outras opções

              Lembre-se: O objetivo é transformar as opções em um roteiro final empolgante e bem informado, mostrando ao usuário que o plano escolhido é o ideal para ele.
            """,
        description="Agente planejador do encontro perfeito",
        tools=[google_search]
    )
    entrada_do_agente_planejador = f"Preferências:{input_inicial}\nLocais buscados: {plano_de_encontro}"
    roteiro = call_agent(planejador, entrada_do_agente_planejador)
    return roteiro

# --- Início da Execução Principal ---

import time # Opcional, apenas para uma pequena pausa/efeito

print("✨ Vamos planejar o encontro perfeito para você! ✨")
print("-" * 30)

# --- Coleta TODAS as Informações do Usuário Aqui ---
# O script vai pausar aqui até que todas as perguntas sejam respondidas

endereco_usuario = input("📍 Primeiro, qual o bairro e cidade de referência para o encontro? (Ex: Centro, São Paulo): ")
horario_encontro = input("⏰ Que horas será o encontro? (Ou digite 'não sei' se quiser uma sugestão): ")
gostos_date_fazer = input("🤸 O que a pessoa com quem você vai sair Gosta de Fazer? (Hobbies, atividades): ")
gostos_usuario_fazer = input("🚶 E o que VOCÊ Gosta de Fazer? (Seus hobbies, atividades preferidas): ")
gostos_usuario_comer = input("🍔 O que VOCÊ Gosta de Comer? (Tipos de culinária, pratos preferidos): ")
gostos_date_comer = input("🍕 E o que a pessoa com quem você vai sair Gosta de Comer? (Tipos de culinária, pratos preferidos): ")
nao_gosta_restricoes = input("🚫 Existe algo que AMBOS NÃO Gostam de Fazer/Comer? Alguma Restrição Alimentar ou Alergia? Descreva tudo aqui: ")
orcamento_passeio = input("💰 Qual o ORÇAMENTO aproximado para o passeio? (Ex: R$ 100, R$ 50-150, Flexível): ")

print("-" * 30)
print("✅ Informações coletadas! Formatando para nossos agentes do amor")
time.sleep(1) # Pequena pausa para efeito

# --- Formatação para o Prompt Inicial da IA ---
# Isso é feito APENAS DEPOIS que todas as informações foram coletadas

input_inicial = f"""Olá, Agente Refinador de Encontros!

Preciso de ajuda para planejar um encontro incrível, baseado nas seguintes informações e preferências:

Local de Referência (Bairro e Cidade): {endereco_usuario}
Horário Desejado para Iniciar (Ou 'não sei'): {horario_encontro}

Preferências de Atividades:
- O que a Outra Pessoa Gosta de Fazer: {gostos_date_fazer}
- O que Eu Gosto de Fazer: {gostos_usuario_fazer}

Preferências Alimentares:
- O que Eu Gosto de Comer: {gostos_usuario_comer}
- O que a Outra Pessoa Gosta de Comer: {gostos_date_comer}

Restrições e Coisas que Ambos Não Gostam:
- Restrições Gerais/Alimentares/Alergias e Atividades/Comidas Não Desejadas: {nao_gosta_restricoes}

Orçamento Total Estimado para o Passeio: {orcamento_passeio}

Por favor, use TODAS estas informações para refinar as opções de encontro, mesclando restaurantes e locais de interesse, e me oferecendo 3 planos distintos com sugestões de rota e presentes, garantindo que cada opção respeite essas condições.

Obrigado! 😊
"""

# --- Executa o Pipeline de Agentes Sequencialmente ---
# Agora que temos todas as entradas, chamamos os agentes um após o outro

print("\n--- Estamos enviando agora para nossos agentes! ---")

# Chama o Agente Buscador
locais_preferenciais = agente_buscador(input_inicial)
# Opcional: Remova ou comente as linhas abaixo para não exibir o resultado intermediário
# print('\n--- Resultado do Agente 1 (Buscador) ---\n')
# display(to_markdown(locais_preferenciais))
# print("-" * 30)

# Chama o Agente Refinador com a entrada inicial e os locais buscados
plano_de_encontro = agente_refinador(input_inicial, locais_preferenciais)
# Opcional: Remova ou comente as linhas abaixo para não exibir o resultado intermediário
# print('\n--- Resultado do Agente 2 (Refinador) ---\n')
# display(to_markdown(plano_de_encontro))
# print("-" * 30)

# Chama o Agente Planejador com a entrada inicial e o plano refinado
roteiro = agente_planejador(input_inicial, plano_de_encontro)

# --- Exibe o Resultado Final ---
print('\n--- Resultado Final (Plano de Encontro Perfeito!) ---\n')
display(to_markdown(roteiro))
print("-" * 30)

✨ Vamos planejar o encontro perfeito para você! ✨
------------------------------
📍 Primeiro, qual o bairro e cidade de referência para o encontro? (Ex: Centro, São Paulo): pindamonhangaba
⏰ Que horas será o encontro? (Ou digite 'não sei' se quiser uma sugestão): me impressione
🤸 O que a pessoa com quem você vai sair Gosta de Fazer? (Hobbies, atividades): me impressione
🚶 E o que VOCÊ Gosta de Fazer? (Seus hobbies, atividades preferidas): me impressione
🍔 O que VOCÊ Gosta de Comer? (Tipos de culinária, pratos preferidos): me impressione
🍕 E o que a pessoa com quem você vai sair Gosta de Comer? (Tipos de culinária, pratos preferidos): me impressione
🚫 Existe algo que AMBOS NÃO Gostam de Fazer/Comer? Alguma Restrição Alimentar ou Alergia? Descreva tudo aqui: veganos
💰 Qual o ORÇAMENTO aproximado para o passeio? (Ex: R$ 100, R$ 50-150, Flexível): me impressione
------------------------------
✅ Informações coletadas! Formatando para nossos agentes do amor

--- Estamos enviando agora para no

> Ah, preparar um encontro especial é uma arte, e eu estou aqui para garantir que você cause a melhor impressão possível! ✨ Depois de analisar cuidadosamente todas as opções, levando em conta o seu desejo de ser surpreendido e as preferências veganas, a **Opção 3: Relaxamento na Natureza e Picnic Vegano** se destaca como a escolha ideal para um momento inesquecível.
> 
> A combinação de um piquenique personalizado com a serenidade do Parque da Cidade oferece a flexibilidade perfeita para criar um encontro que reflita seus gostos e os da sua companhia, tudo isso em um ambiente encantador e relaxante. Além disso, a preparação do piquenique em si pode ser uma atividade divertida e colaborativa, adicionando um toque especial ao encontro. 😉
> 
> **Roteiro Detalhado do Encontro Vegano no Parque da Cidade:**
> 
> 1.  **Início da Tarde (a partir das 14h):** Comece o dia com uma visita a mercados e lojas locais para selecionar os ingredientes veganos mais frescos e saborosos para o seu piquenique. Queijos veganos, frutas da estação, pães artesanais, patês deliciosos e bebidas refrescantes são ótimas opções. 🍓🥖 Prepare tudo com carinho, transformando a montagem do piquenique em um momento divertido e criativo.
> 
> 2.  **Chegada ao Parque da Cidade (por volta das 15h30):** Dirija-se ao Parque da Cidade *(Localizado na Av. N. Sra. do Bom Sucesso, 2500 - Jardim Mariana, Pindamonhangaba - SP, o parque oferece uma ampla área verde, perfeita para relaxar e aproveitar a natureza)*. Escolha um local agradável e tranquilo para estender sua manta e montar o piquenique.
> 
> 3.  **Piquenique Vegano Relaxante (das 16h às 18h):** Desfrute de um delicioso piquenique vegano em meio à natureza. Saboreie cada item, converse, ria e aproveite a companhia um do outro. Que tal levar um baralho ou um jogo de tabuleiro leve para animar o momento? 😄
> 
> 4.  **Exploração do Parque e Atividades ao Ar Livre (das 18h às 19h30):** Após o piquenique, aproveite para explorar as belezas do Parque da Cidade. Caminhe de mãos dadas, ande de bicicleta ou simplesmente relaxe e aprecie a paisagem. *(O Parque da Cidade é um espaço público que oferece diversas opções de lazer e contato com a natureza em Pindamonhangaba)*.
> 
> 5.  **Despedida Romântica (a partir das 19h30):** Para finalizar o encontro com chave de ouro, encontre um local tranquilo no parque para apreciar o pôr do sol juntos. A vista é simplesmente mágica e proporciona um momento perfeito para uma conversa sincera e um abraço apertado. 🥰
> 
> **Sugestão de Presente:**
> 
> Para complementar este encontro especial, sugiro uma cesta de piquenique charmosa, completa com todos os utensílios necessários (pratos, talheres, copos, guardanapos) e uma manta confortável. Você pode encontrar opções lindas em lojas de artigos para casa ou até mesmo montar a sua própria cesta personalizada. 🎁
> 
> Com este plano, o sucesso do seu encontro é garantido! A flexibilidade, o toque pessoal e o ambiente relaxante proporcionam uma experiência única e inesquecível. Tenha um encontro maravilhoso e divirta-se! ❤️
> 


------------------------------
